In [34]:
#
# Working through the example from
# https://github.com/salvadorgarciamunoz/kipet/blob/master/kipet/examples/Ex_7_concentration_input.py
#
# Trying the example with data where B, C conc is available at all time points but A is only available at 
# a subset of time points
#

In [35]:
from __future__ import print_function
from kipet.library.TemplateBuilder import *
from kipet.library.PyomoSimulator import *
from kipet.library.ParameterEstimator import *
from kipet.library.VarianceEstimator import *
from kipet.library.data_tools import *
import matplotlib.pyplot as plt
import os
import sys
import inspect
import six
import pandas as pd

In [37]:
# Creating a dataset from Ex_1_C_data.txt that removed A data from a subset of time points
tmp = pd.read_csv('../data_sets/Ex_1_C_data.txt', index_col = 0, header = None, sep = ' ')
tmp.columns = ['comp', 'conc']
tmp['rownum'] = list(range(tmp.shape[0]))
tmp = tmp[(tmp.comp.isin(['B', 'C'])) | (tmp.rownum % 10 == 0)]
tmp = tmp[['comp', 'conc']]
tmp
tmp.to_csv('../data_sets/Ex_1_C_data_irreg.txt', sep = ' ', header = None)

In [38]:
filename = '../data_sets/Ex_1_C_data_irreg.txt'
C_frame = read_concentration_data_from_txt(filename)

KeyError: (0.0333, 'A')

In [ ]:
C_frame

In [ ]:
fig, ax = plt.subplots()
#ax.scatter(C_frame.index, C_frame['A'], label = 'A')
ax.scatter(C_frame.index, C_frame['B'], label = 'B')
ax.scatter(C_frame.index, C_frame['C'], label = 'C')
ax.legend()

In [ ]:
builder = TemplateBuilder()    
components = {'A':1e-3,'B':0,'C':0}
builder.add_mixture_component(components)
builder.add_parameter('k1',bounds=(0.0,5.0))
builder.add_parameter('k2',bounds=(0.0,1.0))
builder.add_concentration_data(C_frame)

In [ ]:
# define explicit system of ODEs
def rule_odes(m,t):
    exprs = dict()
    exprs['A'] = -m.P['k1']*m.Z[t,'A']
    exprs['B'] = m.P['k1']*m.Z[t,'A']-m.P['k2']*m.Z[t,'B']
    exprs['C'] = m.P['k2']*m.Z[t,'B']
    return exprs
    
builder.set_odes_rule(rule_odes)
opt_model = builder.create_pyomo_model(0.0,10.0)

In [ ]:
#sigmas = {'A':1e-10,'B':1e-11,'C':1e-10}
sigmas = {'B':1e-11,'C':1e-10}

In [ ]:
# and define our parameter estimation problem and discretization strategy
p_estimator = ParameterEstimator(opt_model)
p_estimator.apply_discretization('dae.collocation',nfe=60,ncp=3,scheme='LAGRANGE-RADAU')

In [ ]:
# Again we provide options for the solver, this time providing the scaling that we set above
options = dict()
#options['nlp_scaling_method'] = 'user-scaling'
options['linear_solver'] = 'ma27'

In [ ]:
# finally we run the optimization
results_pyomo = p_estimator.run_opt('ipopt',
                                        variances=sigmas,
                                      tee=True,
                                   solver_opts = options)

In [ ]:
# And display the results
print("The estimated parameters are:")
for k,v in six.iteritems(results_pyomo.P):
    print(k, v)

In [ ]:
results_pyomo.C.plot.line(legend=True)
plt.xlabel("time (s)")
plt.ylabel("Concentration (mol/L)")
plt.title("Concentration Profile")

results_pyomo.Z.plot.line(legend=True)
plt.xlabel("time (s)")
plt.ylabel("Concentration (mol/L)")
plt.title("Concentration Profile")


plt.show()